In [51]:
import pandas as pd
import geopandas as gp
import matplotlib as pl
from shapely.geometry import Point
import matplotlib.cm as cm

%pylab inline

Populating the interactive namespace from numpy and matplotlib


`%matplotlib` prevents importing * from pylab and numpy


In [29]:
#Stations file
stations = pd.read_csv('../data/external/citibike_data/citi_bike_station_info.csv')
geometry = gp.GeoSeries([Point(xy) for xy in zip(stations.Longitude, stations.Latitude)])
geo_stations = gp.GeoDataFrame(stations, geometry=geometry)
geo_stations.crs = {'init' :'epsg:4326'}
geo_stations.drop(['Location', 'Latitude', 'Longitude'], axis = 1, inplace = True)
geo_stations.rename(columns={'Station_id' : 'station_id'}, inplace=True)
geo_stations.head()

,station_id,Station_Name,geometry
0,72,W 52 St & 11 Ave,POINT (-73.99392888 40.76727216)
1,79,Franklin St & W Broadway,POINT (-74.00666661 40.71911552)
2,82,St James Pl & Pearl St,POINT (-74.00016545 40.71117416)
3,83,Atlantic Ave & Fort Greene Pl,POINT (-73.97632328 40.68382604)
4,116,W 17 St & 8 Ave,POINT (-74.00149746 40.74177603)


In [30]:
#Complied master file
master = pd.read_csv('../data/processed/master_norm.csv')
master.head()

,station_id,ridership_0115,ridership_0215,ridership_0315,ridership_0415,ridership_0515,ridership_0615,ridership_0715,ridership_0815,ridership_0915,...,ridership_1215,avg_ridership_2015,bike_lane_score,street_quality_score,tree_score,traffic_volume,median_hh_income,pop_density,park,subway_entrance
0,72,-0.361416,-0.259163,-0.019305,-0.017391,0.013852,-0.044803,-0.053536,0.290089,0.380089,...,-0.037283,0.134845,-1.037727,0.790673,-0.554668,0.001493,1.069154e-01,0.791144,1.0,0.0
1,79,-0.485013,-0.599695,-0.423754,-0.190652,-0.278161,-0.223134,-0.296086,-0.030448,0.089412,...,-0.087781,-0.153108,0.722092,1.584334,-0.904716,-0.424358,2.130764e-15,0.218505,0.0,1.0
2,82,-0.924468,-0.738256,-0.852390,-0.930948,-0.938365,-0.991566,-0.960664,-0.681822,-0.728118,...,-0.721558,-0.924446,-1.037727,-0.135265,0.240854,0.155045,-3.670914e-01,-0.170810,1.0,0.0
3,83,-1.013732,-1.005985,-1.020350,-0.961663,-0.921605,-0.966611,-0.744548,-0.443273,-0.577905,...,-0.718631,-0.849354,-1.037727,0.096220,-1.334880,2.144695,-3.877741e-02,-1.079345,0.0,0.0
4,116,1.454770,1.415314,1.430531,-0.029204,-0.322344,1.012182,1.117474,1.390545,1.661106,...,1.454217,1.256217,1.308698,1.485126,0.813900,0.086690,5.403333e-01,0.581773,1.0,0.0


In [31]:
master.columns

Index([u'station_id', u'ridership_0115', u'ridership_0215', u'ridership_0315',
       u'ridership_0415', u'ridership_0515', u'ridership_0615',
       u'ridership_0715', u'ridership_0815', u'ridership_0915',
       u'ridership_1015', u'ridership_1115', u'ridership_1215',
       u'avg_ridership_2015', u'bike_lane_score', u'street_quality_score',
       u'tree_score', u'traffic_volume', u'median_hh_income', u'pop_density',
       u'park', u'subway_entrance'],
      dtype='object')

In [32]:
#Dataframe of ridership only
ridership = master.copy()
ridership.drop([u'bike_lane_score', u'park',
       u'street_quality_score', u'subway_entrance', u'tree_score',
       u'traffic_volume', u'median_hh_income', u'pop_density'], axis = 1, inplace = True)
ridership.head()

,station_id,ridership_0115,ridership_0215,ridership_0315,ridership_0415,ridership_0515,ridership_0615,ridership_0715,ridership_0815,ridership_0915,ridership_1015,ridership_1115,ridership_1215,avg_ridership_2015
0,72,-0.361416,-0.259163,-0.019305,-0.017391,0.013852,-0.044803,-0.053536,0.290089,0.380089,0.450563,0.130947,-0.037283,0.134845
1,79,-0.485013,-0.599695,-0.423754,-0.190652,-0.278161,-0.223134,-0.296086,-0.030448,0.089412,0.014532,-0.017566,-0.087781,-0.153108
2,82,-0.924468,-0.738256,-0.852390,-0.930948,-0.938365,-0.991566,-0.960664,-0.681822,-0.728118,-0.757243,-0.766095,-0.721558,-0.924446
3,83,-1.013732,-1.005985,-1.020350,-0.961663,-0.921605,-0.966611,-0.744548,-0.443273,-0.577905,-0.732535,-0.726134,-0.718631,-0.849354
4,116,1.454770,1.415314,1.430531,-0.029204,-0.322344,1.012182,1.117474,1.390545,1.661106,1.554691,1.402552,1.454217,1.256217


In [33]:
ridership = pd.merge(ridership, geo_stations, on = 'station_id', how = 'inner')
ridership.head()

,station_id,ridership_0115,ridership_0215,ridership_0315,ridership_0415,ridership_0515,ridership_0615,ridership_0715,ridership_0815,ridership_0915,ridership_1015,ridership_1115,ridership_1215,avg_ridership_2015,Station_Name,geometry
0,72,-0.361416,-0.259163,-0.019305,-0.017391,0.013852,-0.044803,-0.053536,0.290089,0.380089,0.450563,0.130947,-0.037283,0.134845,W 52 St & 11 Ave,POINT (-73.99392888 40.76727216)
1,79,-0.485013,-0.599695,-0.423754,-0.190652,-0.278161,-0.223134,-0.296086,-0.030448,0.089412,0.014532,-0.017566,-0.087781,-0.153108,Franklin St & W Broadway,POINT (-74.00666661 40.71911552)
2,82,-0.924468,-0.738256,-0.852390,-0.930948,-0.938365,-0.991566,-0.960664,-0.681822,-0.728118,-0.757243,-0.766095,-0.721558,-0.924446,St James Pl & Pearl St,POINT (-74.00016545 40.71117416)
3,83,-1.013732,-1.005985,-1.020350,-0.961663,-0.921605,-0.966611,-0.744548,-0.443273,-0.577905,-0.732535,-0.726134,-0.718631,-0.849354,Atlantic Ave & Fort Greene Pl,POINT (-73.97632328 40.68382604)
4,116,1.454770,1.415314,1.430531,-0.029204,-0.322344,1.012182,1.117474,1.390545,1.661106,1.554691,1.402552,1.454217,1.256217,W 17 St & 8 Ave,POINT (-74.00149746 40.74177603)


In [34]:
ridership = gp.GeoDataFrame(ridership)
ridership.to_file('../data/processed/ridership_norm')

In [35]:
#Dataframe of the matrics
regressors = master.copy()
regressors.drop([u'ridership_0115', u'ridership_0215', u'ridership_0315',
       u'ridership_0415', u'ridership_0515', u'ridership_0615',
       u'ridership_0715', u'ridership_0815', u'ridership_0915',
       u'ridership_1015', u'ridership_1115', u'ridership_1215',
       u'avg_ridership_2015'], axis = 1, inplace = True)
regressors.head()

,station_id,bike_lane_score,street_quality_score,tree_score,traffic_volume,median_hh_income,pop_density,park,subway_entrance
0,72,-1.037727,0.790673,-0.554668,0.001493,1.069154e-01,0.791144,1.0,0.0
1,79,0.722092,1.584334,-0.904716,-0.424358,2.130764e-15,0.218505,0.0,1.0
2,82,-1.037727,-0.135265,0.240854,0.155045,-3.670914e-01,-0.170810,1.0,0.0
3,83,-1.037727,0.096220,-1.334880,2.144695,-3.877741e-02,-1.079345,0.0,0.0
4,116,1.308698,1.485126,0.813900,0.086690,5.403333e-01,0.581773,1.0,0.0


In [36]:
regressors = pd.merge(regressors, geo_stations, on = 'station_id', how = 'inner')
regressors.head()

,station_id,bike_lane_score,street_quality_score,tree_score,traffic_volume,median_hh_income,pop_density,park,subway_entrance,Station_Name,geometry
0,72,-1.037727,0.790673,-0.554668,0.001493,1.069154e-01,0.791144,1.0,0.0,W 52 St & 11 Ave,POINT (-73.99392888 40.76727216)
1,79,0.722092,1.584334,-0.904716,-0.424358,2.130764e-15,0.218505,0.0,1.0,Franklin St & W Broadway,POINT (-74.00666661 40.71911552)
2,82,-1.037727,-0.135265,0.240854,0.155045,-3.670914e-01,-0.170810,1.0,0.0,St James Pl & Pearl St,POINT (-74.00016545 40.71117416)
3,83,-1.037727,0.096220,-1.334880,2.144695,-3.877741e-02,-1.079345,0.0,0.0,Atlantic Ave & Fort Greene Pl,POINT (-73.97632328 40.68382604)
4,116,1.308698,1.485126,0.813900,0.086690,5.403333e-01,0.581773,1.0,0.0,W 17 St & 8 Ave,POINT (-74.00149746 40.74177603)


In [37]:
regressors.shape

(452, 11)

In [11]:
regressors = gp.GeoDataFrame(ridership)
regressors.to_file('../data/processed/regressors_norm')

In [12]:
master_shape = pd.merge(master, geo_stations, on = 'station_id', how = 'inner')
master_shape = gp.GeoDataFrame(master_shape)
master_shape.head()

,station_id,ridership_0115,ridership_0215,ridership_0315,ridership_0415,ridership_0515,ridership_0615,ridership_0715,ridership_0815,ridership_0915,...,bike_lane_score,street_quality_score,tree_score,traffic_volume,median_hh_income,pop_density,park,subway_entrance,Station_Name,geometry
0,72,-0.361416,-0.259163,-0.019305,-0.017391,0.013852,-0.044803,-0.053536,0.290089,0.380089,...,-1.037727,0.790673,-0.554668,0.001493,1.069154e-01,0.791144,1.0,0.0,W 52 St & 11 Ave,POINT (-73.99392888 40.76727216)
1,79,-0.485013,-0.599695,-0.423754,-0.190652,-0.278161,-0.223134,-0.296086,-0.030448,0.089412,...,0.722092,1.584334,-0.904716,-0.424358,2.130764e-15,0.218505,0.0,1.0,Franklin St & W Broadway,POINT (-74.00666661 40.71911552)
2,82,-0.924468,-0.738256,-0.852390,-0.930948,-0.938365,-0.991566,-0.960664,-0.681822,-0.728118,...,-1.037727,-0.135265,0.240854,0.155045,-3.670914e-01,-0.170810,1.0,0.0,St James Pl & Pearl St,POINT (-74.00016545 40.71117416)
3,83,-1.013732,-1.005985,-1.020350,-0.961663,-0.921605,-0.966611,-0.744548,-0.443273,-0.577905,...,-1.037727,0.096220,-1.334880,2.144695,-3.877741e-02,-1.079345,0.0,0.0,Atlantic Ave & Fort Greene Pl,POINT (-73.97632328 40.68382604)
4,116,1.454770,1.415314,1.430531,-0.029204,-0.322344,1.012182,1.117474,1.390545,1.661106,...,1.308698,1.485126,0.813900,0.086690,5.403333e-01,0.581773,1.0,0.0,W 17 St & 8 Ave,POINT (-74.00149746 40.74177603)


In [13]:
master_shape.to_file('../data/processed/master_shape_norm')

In [42]:
#Checking silhouette score
from sklearn.metrics import silhouette_samples, silhouette_score
from sklearn.cluster import KMeans, DBSCAN
import matplotlib as plt
X=regressors.iloc[:, range(1,9)]
X.head()

,bike_lane_score,street_quality_score,tree_score,traffic_volume,median_hh_income,pop_density,park,subway_entrance
0,-1.037727,0.790673,-0.554668,0.001493,1.069154e-01,0.791144,1.0,0.0
1,0.722092,1.584334,-0.904716,-0.424358,2.130764e-15,0.218505,0.0,1.0
2,-1.037727,-0.135265,0.240854,0.155045,-3.670914e-01,-0.170810,1.0,0.0
3,-1.037727,0.096220,-1.334880,2.144695,-3.877741e-02,-1.079345,0.0,0.0
4,1.308698,1.485126,0.813900,0.086690,5.403333e-01,0.581773,1.0,0.0


In [43]:
range_n_clusters = range(2,10)
for n_clusters in range_n_clusters:
    km = KMeans(n_clusters=n_clusters, random_state=324)
    cluster_labels = km.fit_predict(X)
    silhouette_avg = silhouette_score(X, cluster_labels)
    print("For n_clusters ={},".format(n_clusters)+" the average silhouette_score is :{}".format(silhouette_avg))

For n_clusters =2, the average silhouette_score is :0.162248720831
For n_clusters =3, the average silhouette_score is :0.144269282317
For n_clusters =4, the average silhouette_score is :0.151254316705
For n_clusters =5, the average silhouette_score is :0.155389280954
For n_clusters =6, the average silhouette_score is :0.174429533746
For n_clusters =7, the average silhouette_score is :0.174242858852
For n_clusters =8, the average silhouette_score is :0.164952646611
For n_clusters =9, the average silhouette_score is :0.167414915132


In [91]:
dbscan_cluster = KMeans(6).fit(X)
regressors['dbscan_labels'] = dbscan_cluster.labels_

In [92]:
regressors.dbscan_labels.unique()

array([1, 3, 0, 5, 4, 2])

In [93]:
regressors = gp.GeoDataFrame(regressors)
regressors.to_file('../data/processed/KMeans')

In [94]:
master.head()

,station_id,ridership_0115,ridership_0215,ridership_0315,ridership_0415,ridership_0515,ridership_0615,ridership_0715,ridership_0815,ridership_0915,...,avg_ridership_2015,bike_lane_score,street_quality_score,tree_score,traffic_volume,median_hh_income,pop_density,park,subway_entrance,cluster
0,72,-0.361416,-0.259163,-0.019305,-0.017391,0.013852,-0.044803,-0.053536,0.290089,0.380089,...,0.134845,-1.037727,0.790673,-0.554668,0.001493,1.069154e-01,0.791144,1.0,0.0,1
1,79,-0.485013,-0.599695,-0.423754,-0.190652,-0.278161,-0.223134,-0.296086,-0.030448,0.089412,...,-0.153108,0.722092,1.584334,-0.904716,-0.424358,2.130764e-15,0.218505,0.0,1.0,2
2,82,-0.924468,-0.738256,-0.852390,-0.930948,-0.938365,-0.991566,-0.960664,-0.681822,-0.728118,...,-0.924446,-1.037727,-0.135265,0.240854,0.155045,-3.670914e-01,-0.170810,1.0,0.0,1
3,83,-1.013732,-1.005985,-1.020350,-0.961663,-0.921605,-0.966611,-0.744548,-0.443273,-0.577905,...,-0.849354,-1.037727,0.096220,-1.334880,2.144695,-3.877741e-02,-1.079345,0.0,0.0,3
4,116,1.454770,1.415314,1.430531,-0.029204,-0.322344,1.012182,1.117474,1.390545,1.661106,...,1.256217,1.308698,1.485126,0.813900,0.086690,5.403333e-01,0.581773,1.0,0.0,5


In [96]:
master['cluster'] = regressors.dbscan_labels

In [98]:
master = gp.GeoDataFrame(master, geometry = stations.geometry)
master.to_file('../data/processed/master_cluster')

In [100]:
master

,station_id,ridership_0115,ridership_0215,ridership_0315,ridership_0415,ridership_0515,ridership_0615,ridership_0715,ridership_0815,ridership_0915,...,bike_lane_score,street_quality_score,tree_score,traffic_volume,median_hh_income,pop_density,park,subway_entrance,cluster,geometry
0,72,-0.361416,-0.259163,-0.019305,-0.017391,0.013852,-0.044803,-0.053536,0.290089,0.380089,...,-1.037727,7.906729e-01,-0.554668,1.493426e-03,1.069154e-01,0.791144,1.0,0.0,1,POINT (-73.99392888 40.76727216)
1,79,-0.485013,-0.599695,-0.423754,-0.190652,-0.278161,-0.223134,-0.296086,-0.030448,0.089412,...,0.722092,1.584334e+00,-0.904716,-4.243585e-01,2.130764e-15,0.218505,0.0,1.0,3,POINT (-74.00666661 40.71911552)
2,82,-0.924468,-0.738256,-0.852390,-0.930948,-0.938365,-0.991566,-0.960664,-0.681822,-0.728118,...,-1.037727,-1.352646e-01,0.240854,1.550452e-01,-3.670914e-01,-0.170810,1.0,0.0,1,POINT (-74.00016545 40.71117416)
3,83,-1.013732,-1.005985,-1.020350,-0.961663,-0.921605,-0.966611,-0.744548,-0.443273,-0.577905,...,-1.037727,9.621976e-02,-1.334880,2.144695e+00,-3.877741e-02,-1.079345,0.0,0.0,0,POINT (-73.97632328 40.68382604)
4,116,1.454770,1.415314,1.430531,-0.029204,-0.322344,1.012182,1.117474,1.390545,1.661106,...,1.308698,1.485126e+00,0.813900,8.669016e-02,5.403333e-01,0.581773,1.0,0.0,5,POINT (-74.00149746 40.74177603)
5,119,-1.254059,-1.226744,-1.285056,-1.449156,-1.400507,-1.427774,-1.431850,-1.050975,-1.145080,...,1.308698,7.906729e-01,-1.246834,4.119876e+00,-1.155971e+00,-0.951429,1.0,0.0,0,POINT (-73.97803415 40.69608941)
6,120,-1.272942,-1.215001,-1.168156,-1.271957,-1.237995,-1.219809,-1.244489,-0.897711,-0.986891,...,1.308698,-1.352646e-01,0.466704,-6.281717e-01,2.130764e-15,-0.309993,0.0,0.0,3,POINT (-73.95928168 40.68676793)
7,127,0.876269,0.750690,0.730471,1.328531,2.345387,2.315086,1.701356,1.643926,1.938047,...,1.308698,3.277041e-01,2.579971,-5.367285e-01,2.130764e-15,0.081575,0.0,0.0,4,POINT (-74.00674436 40.73172428)
8,128,1.003299,0.898645,0.879620,1.262377,1.630843,1.547693,1.357705,1.568118,1.022148,...,0.722092,-5.982333e-01,1.800732,-5.687260e-02,2.130764e-15,-0.362684,1.0,0.0,4,POINT (-74.00297088000001 40.72710258)
9,137,-0.382015,-0.334315,-0.284011,-0.429279,-0.433563,-0.533002,-0.484839,0.000040,-0.028011,...,-1.037727,-5.982333e-01,-0.071971,-3.345889e-01,8.516327e-01,-1.698444,0.0,0.0,1,POINT (-73.97292400000001 40.761628)


In [99]:
master.avg_ridership_2015.sort_values(ascending=False)

273    4.768187
75     4.222066
196    3.708776
253    3.650338
189    3.225451
72     2.895952
15     2.690493
71     2.553739
170    2.533742
142    2.419313
104    2.417820
246    2.382662
234    2.366485
255    2.288468
218    2.256772
309    2.170435
205    2.146521
132    2.124733
282    2.123360
289    2.064743
150    2.045701
300    2.008334
7      1.993351
148    1.892353
155    1.887637
401    1.791426
276    1.684148
207    1.675970
182    1.650243
22     1.643199
         ...   
40    -1.108118
198   -1.108954
128   -1.114267
164   -1.121668
185   -1.124892
27    -1.132950
183   -1.138860
167   -1.148649
127   -1.150261
138   -1.155096
118   -1.169840
64    -1.178256
66    -1.184106
117   -1.198731
145   -1.204640
186   -1.214131
11    -1.214728
6     -1.219145
184   -1.219503
79    -1.221951
173   -1.223904
43    -1.247439
175   -1.254423
295   -1.302654
197   -1.316861
144   -1.324382
204   -1.330411
73    -1.334410
299   -1.363062
5     -1.401265
Name: avg_ridership_2015